In [1]:
import numpy as np
import matplotlib.pyplot as plt
import stock
from IPython.display import display
import ipywidgets as widgets
from ipywidgets import Layout, Button, Box
import xueqiu
import kline
import status
from datetime import date,timedelta

In [2]:
def isRasing(a):
    #macd 在两日日内有翻红趋势，energy从1以下崛起到大于5，volumeJ趋势向上,close趋势向上
    #0 id , 1 close , 2 volume , 3 volumema20 , 4 macd , 5 energy ,6 volumeJ
    dMACD = a[2][4]-a[1][4]
    dVolumeJ = a[2][6]-a[1][6]
    dClose = a[2][1]-a[1][1]
    if a[2][4]<0 and a[2][4] + 2*dMACD >= 0 and dVolumeJ>0 and dClose>0 and a[2][5]>=5 and a[1][5]<5:
        return True
    return False

#找到有崛起继续的个股id , dd是搜索指定日期
def searchRasingCompanyStatus(dd):
    lastday = stock.query("""select date from company_status where id=8828 order by date desc limit 20""")
    d = lastday[-1][0]
    
    for i in range(len(lastday)):
        if str(lastday[i][0])==dd:
            endDate = lastday[i][0]
            beginDate = lastday[i+2][0]
    cs = np.array(stock.query("""select id,close,volume,volumema20,macd,energy,volumeJ from company_status where date>='%s' and date<='%s'"""%(stock.dateString(beginDate),stock.dateString(endDate))))
    stock.closedb()
    idds = {}
    for i in range(len(cs)):
        if cs[i][0] not in idds:
            idds[cs[i][0]] = []
        idds[cs[i][0]].append(cs[i])
    rasing = []
    for c in idds:
        if len(idds[c])==3 and isRasing(idds[c]):
            rasing.append(int(c))
    return rasing

"""
按分类列出崛起的股票的数量与列表
"""
def DayRasingCategoryList():
    status.update_status() #更新公司日状态
    output = widgets.Output()    
    items_layout = Layout( width='auto')     # override the default width of the button to 'auto' to let the button grow
    box_layout = Layout(display='flex',
                        flex_flow='wrap',
                        align_items='stretch',
                        border='solid',
                        width='100%')
    #可以提前准备的数据
    categorys = stock.query("""select id,name from category""")
    companys = stock.query("""select id,code,name,category from company""")
    
    def onCatsList(E):
        rasing = searchRasingCompanyStatus(E.description)

        stock.closedb()
        cats = {}
        rasingCompany = []
        for c in companys:
            if c[0] in rasing:
                rasingCompany.append(c)
        # cats = {"id":{"name":,"ls":,"count":0}}
        for c in categorys:
            if c[0] not in cats:
                cats[c[0]] = {"name":c[1],"ls":[],"count":0}
        for c in companys:
            if c[0] in rasing and c[3] in cats:
                cats[c[3]]['ls'].append(c)
                cats[c[3]]['count'] += 1
        #计算分类中的崛起数量，不列出那些没有崛起的分类
        items = []
        def onClick(e):
            output.clear_output(wait=True)
            key = int(e.tooltip)
            with output:
                display(box)
                for c in cats[key]['ls']:
                    kline.Plote(c[1],config={"index":True,"markpos":date.fromisoformat(E.description)}).show(figsize=(32,15))
                
        for c in cats:
            if cats[c]["count"]>0:
                s = "%s %d"%(cats[c]["name"],cats[c]["count"])
                but = widgets.Button(
                    description=s,
                    disabled=False,
                    button_style='',
                    tooltip=str(c))
                but.on_click(onClick)
                items.append(but)
        box = Box(children=items, layout=box_layout)
        output.clear_output(wait=True)
        with output:
            display(box)

    items = []
    dates = stock.query('select date from company_status where id=8828 order by date desc limit 10')
    for d in dates:
        but = widgets.Button(
            description=str(d[0]),
            disabled=False,
            button_style='')
        but.on_click(onCatsList)
        items.append(but)
    box = Box(children=items, layout=box_layout)
    display(box,output)

# 日线崛起态势的股票进行显示

In [3]:
DayRasingCategoryList()

Box(children=(Button(description='2020-01-17', style=ButtonStyle()), Button(description='2020-01-16', style=Bu…

Output()